Lasso 回归（Lasso Regression）
Lasso回归（Least Absolute Shrinkage and Selection Operator）是一种线性回归方法，通过在损失函数中引入 L1 正则化项 来防止模型过拟合。它不仅能提升模型泛化能力，还能实现特征选择。

1. 目标函数 (Objective Function)
Lasso回归的目标是最小化以下损失函数：
$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})^2 + \frac{\lambda}{2m} \sum_{j=1}^{n} |\theta_j| $$
其中：
$ h_\theta(x) $：假设函数，即预测值；
$ m $：样本数量；
$ n $：特征数量（不包括偏置项）；
$ \lambda $：正则化参数，控制惩罚强度（$\lambda > 0$）；
$ \theta_j $：模型参数（权重）；
$ x^{(i)} $：第 $ i $ 个样本的输入特征；
$ y^{(i)} $：第 $ i $ 个样本的真实值。
假设函数形式
对于多元线性回归，假设函数为：
$$ h_\theta(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n = \sum_{j=0}^{n} \theta_j x_j $$
其中 $ x_0 = 1 $，作为偏置项。

2. 求解方式（无解析解）
与岭回归不同，Lasso回归的目标函数由于含有绝对值项（不可导），无法直接求出闭式解。通常采用以下优化方法进行求解：
梯度下降法（Gradient Descent）
坐标下降法（Coordinate Descent）
最小角回归（LARS, Least Angle Regression）

3. 梯度下降更新规则（Gradient Descent Update Rule）
由于L1正则项在零点不可导，通常使用次梯度（subgradient）来处理。参数更新规则如下：
对于 $ j = 0 $（偏置项）：
$$ \theta_0 := \theta_0 - \alpha \cdot \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) \cdot x_0^{(i)} $$
对于 $ j = 1, 2, ..., n $（非偏置项）：
$$ \theta_j := \theta_j - \alpha \cdot \left( \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) \cdot x_j^{(i)} + \frac{\lambda}{m} \cdot \text{sign}(\theta_j) \right) $$
其中：
$ \alpha $：学习率；
$ \text{sign}(\theta_j) $：符号函数，表示 $ \theta_j $ 的正负号（当 $ \theta_j = 0 $ 时可取任意值）；
其余变量定义同上。

4. 向量化形式（Vectorized Form）
在实际代码实现中，可以将参数更新写成向量化形式以提高效率：
$$ \theta := \theta - \alpha \cdot \left( \frac{1}{m} X^T (X\theta - y) + \frac{\lambda}{m} \cdot \text{sign}(\theta[1:]) \right) $$
注意：偏置项 $ \theta_0 $ 不参与正则化。

In [ ]:
import numpy as np

def lasso_regression_gradient_descent(X, y, theta, learning_rate=0.01, num_iterations=1000, lambda_=1.0):
    m = X.shape[0]  # 样本数量
    costs = np.zeros(num_iterations)

    for i in range(num_iterations):
        predictions = X @ theta
        errors = predictions - y
        gradient = (X.T @ errors) / m
        sign_theta = np.sign(theta)
        sign_theta[0] = 0  # 偏置项不加正则化
        gradient += (lambda_ / m) * sign_theta
        theta -= learning_rate * gradient
        costs[i] = np.sum(errors**2) / (2*m) + (lambda_ / (2*m)) * np.sum(np.abs(theta[1:]))
    return theta, costs
